# Task 1

Check out this official repository with many examples of Keras implementations of various sorts of
deep neural networks here. We recommend cloning this repository and trying to get some of these
examples running on your system (or Colab/DeepNote). In particular, experiment with mnist mlp.py
and mnist cnn.py scripts which show you how to build simple neural networks for the MNIST dataset
(useful for the next task

Next, take the two well-known datasets: Fashion MNIST (introduced in Ch 10, p. 298) and CIFAR-10.
The first dataset contains 2D (grayscale) images of size 28x28, split into 10 categories; 60,000 images
for training and 10,000 for testing, while the latter contains 32x32x3 RGB images (50,000/10,000
train/test). Apply two reference networks on the fashion MNIST datase

## (a) MLP

initializations, activations, optimizers (and
their hyperparameters), regularizations (L1, L2, Dropout, no Dropout). You may also experiment
with changing the architecture of both networks: adding/removing layers, number of convolutional
filters, their sizes, etc.

In [ ]:
import tensorflow as tf

2025-11-08 12:21:57.771730: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-08 12:22:09.578129: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-08 12:23:08.170003: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [ ]:
# Check number of available GPUs
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
# Use TensorFlow's bundled Keras to ensure compatibility with GPUs
from tensorflow import keras
import os
from sklearn.preprocessing import StandardScaler

random_state = 900
keras.utils.set_random_seed(random_state)

# Try to enable memory growth for all GPUs so TF doesn't reserve all GPU memory upfront
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print('Enabled memory growth for GPUs:', gpus)
    except Exception as e:
        print('Could not set memory growth:', e)
else:
    print('No GPU devices found by TensorFlow')

Enabled memory growth for GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Model 1

Below is our first attempt at building a neural network. We use the SGD optimizer.

In [ ]:
# Import FashionMNIST data
fashion_mnist= keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
print(X_train_full.shape, y_train_full.shape)

# Preprocess data
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_test = X_test / 255.0
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)


# Choose hyperparameters
seed = 900
activation_function = 'relu6'
kernel_ini = keras.initializers.Orthogonal(gain = 1.0, seed = seed)
kernel_reg = keras.regularizers.l2(0.0001)
bias_ini = keras.initializers.Zeros()
shape = [28, 28]

# Build model
model1 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=shape),
    keras.layers.Dense(400, kernel_initializer = kernel_ini,
    activation=activation_function, kernel_regularizer=kernel_reg),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(200, kernel_initializer = kernel_ini,
    activation=activation_function),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(100, kernel_initializer = kernel_ini,
    activation=activation_function),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(50, kernel_initializer = kernel_ini,
    activation=activation_function),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(10, kernel_initializer = kernel_ini,
    activation="softmax")
])
early_stopping = keras.callbacks.EarlyStopping(
    patience=6,
    restore_best_weights=True
)

model1.summary()


model1.compile(loss="sparse_categorical_crossentropy",
optimizer=keras.optimizers.SGD(learning_rate=5e-3),
metrics=["accuracy",
        #   tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
          ])
model1.fit(
    X_train, y_train,
    epochs=50,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping]
)

#evaluate the model on the test set
test_loss, test_acc = model1.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_22 (Flatten)            │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_107 (Dense)               │ (None, 400)            │       314,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_85 (Dropout)            │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_108 (Dense)               │ (None, 200)            │        80,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_86 (Dropout)            │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_109 (Dense)               │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_87 (Dropout)            │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_110 (Dense)               │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_88 (Dropout)            │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_111 (Dense)               │ (None, 10)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 419,860 (1.60 MB)

 Trainable params: 419,860 (1.60 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.5685 - loss: 1.3064 - val_accuracy: 0.7602 - val_loss: 0.7455
Epoch 2/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7486 - loss: 0.7435 - val_accuracy: 0.8166 - val_loss: 0.5876
Epoch 3/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7927 - loss: 0.6289 - val_accuracy: 0.8400 - val_loss: 0.5215
Epoch 4/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8143 - loss: 0.5720 - val_accuracy: 0.8474 - val_loss: 0.4865
Epoch 5/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8276 - loss: 0.5335 - val_accuracy: 0.8574 - val_loss: 0.4570
Epoch 6/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8374 - loss: 0.5023 - val_accuracy: 0.8620 - val_loss: 0.4373
Epoch 7/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8442 - loss: 0.4828 - val_accuracy: 0.8672 - val_loss: 0.4229
Epoch 8/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8493 - loss: 0.4634 -

In [9]:
#evaluate the model on the test set
test_loss, test_acc = model1.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)
#0.8834999799728394

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8618 - loss: 0.4676
Test accuracy: 0.8618000149726868
Test loss: 0.46763134002685547


#### Application to CIFAR10
We apply the model with chosen hyperparameters on the CIFAR10 set.

In [ ]:
# Import CIFAR10 dataset
CIFAR10 = tf.keras.datasets.cifar10


# Preprocess data
(X_train_full, y_train_full), (X_test, y_test) = CIFAR10.load_data()
print(X_train_full.shape, y_train_full.shape)

X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_test = X_test / 255.0
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)


# Choose hyperparameters
seed = 900
activation_function = 'relu6'
kernel_ini = keras.initializers.Orthogonal(gain = 1.0, seed = seed)
bias_ini = keras.initializers.Zeros()
shape = [32,32,3]


# Build model
model1 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=shape),
    keras.layers.Dense(400, kernel_initializer = kernel_ini,
    activation=activation_function, kernel_regularizer=keras.regularizers.l2(0.0001)),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(200, kernel_initializer = kernel_ini,
    activation=activation_function),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(100, kernel_initializer = kernel_ini,
    activation=activation_function),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(50, kernel_initializer = kernel_ini,
    activation=activation_function),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(10, kernel_initializer = kernel_ini,
    activation="softmax")
])
early_stopping = keras.callbacks.EarlyStopping(
    patience=6,
    restore_best_weights=True
)

model1.summary()


model1.compile(loss="sparse_categorical_crossentropy",
optimizer=keras.optimizers.SGD(learning_rate=5e-3),
metrics=["accuracy",
        #   tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
          ])
model1.fit(
    X_train, y_train,
    epochs=50,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping]
)

#evaluate the model on the test set
test_loss, test_acc = model1.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

/vol/home/s2358441/.conda/envs/IDL/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_24 (Flatten)            │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_117 (Dense)               │ (None, 400)            │     1,229,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_93 (Dropout)            │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_118 (Dense)               │ (None, 200)            │        80,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_94 (Dropout)            │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_119 (Dense)               │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_95 (Dropout)            │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_120 (Dense)               │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_96 (Dropout)            │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_121 (Dense)               │ (None, 10)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,335,060 (5.09 MB)

 Trainable params: 1,335,060 (5.09 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.2290 - loss: 2.1300 - val_accuracy: 0.3240 - val_loss: 1.9446
Epoch 2/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.3124 - loss: 1.9376 - val_accuracy: 0.3514 - val_loss: 1.8714
Epoch 3/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.3457 - loss: 1.8571 - val_accuracy: 0.3704 - val_loss: 1.7985
Epoch 4/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.3659 - loss: 1.7963 - val_accuracy: 0.3734 - val_loss: 1.7994
Epoch 5/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.3854 - loss: 1.7520 - val_accuracy: 0.3932 - val_loss: 1.7423
Epoch 6/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4014 - loss: 1.7138 - val_accuracy: 0.4008 - val_loss: 1.7085
Epoch 7/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4109 - loss: 1.6806 - val_accuracy: 0.4206 - val_loss: 1.6449
Epoch 8/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4245 - loss: 1.6487 -

### Model 2

In [15]:
# Import FashionMNIST data
fashion_mnist= keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
print(X_train_full.shape, y_train_full.shape)


# Preprocess data
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_test = X_test / 255.0
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)


# Choose hyperparameters
seed = 900
activation_function = 'leaky_relu'

kernel_ini = "glorot_uniform"#keras.initializers.Orthogonal(gain = 1.0, seed = seed)
bias_ini = keras.initializers.Zeros()
shape = [28, 28]
learning_rate = 0.001


# Build model
model1 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=shape),
    keras.layers.Dense(400, kernel_initializer = kernel_ini,
    activation=activation_function),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(200, kernel_initializer = kernel_ini,
    activation=activation_function),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(100, kernel_initializer = kernel_ini,
    activation=activation_function, kernel_regularizer=kernel_reg),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(50, kernel_initializer = kernel_ini,
    activation=activation_function),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(10, kernel_initializer = kernel_ini,
    activation="softmax")
])
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True
)

model1.summary()


model1.compile(loss="sparse_categorical_crossentropy",
optimizer=keras.optimizers.AdamW(learning_rate=learning_rate),
metrics=["accuracy",
        #   tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
          ])
model1.fit(
    X_train, y_train,
    epochs=50,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping]
)

# Evaluate the model on the test set
test_loss, test_acc = model1.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

(60000, 28, 28) (60000,)
(55000, 28, 28) (55000,)
(5000, 28, 28) (5000,)


/vol/home/s2358441/.conda/envs/IDL/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_11 (Flatten)            │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_55 (Dense)                │ (None, 400)            │       314,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_44 (Dropout)            │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_56 (Dense)                │ (None, 200)            │        80,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_45 (Dropout)            │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_57 (Dense)                │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_46 (Dropout)            │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_58 (Dense)                │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_47 (Dropout)            │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_59 (Dense)                │ (None, 10)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 419,860 (1.60 MB)

 Trainable params: 419,860 (1.60 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - accuracy: 0.8036 - loss: 0.5515 - val_accuracy: 0.8560 - val_loss: 0.4168
Epoch 2/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8500 - loss: 0.4216 - val_accuracy: 0.8598 - val_loss: 0.4036
Epoch 3/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8620 - loss: 0.3881 - val_accuracy: 0.8678 - val_loss: 0.3862
Epoch 4/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8709 - loss: 0.3642 - val_accuracy: 0.8750 - val_loss: 0.3611
Epoch 5/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8767 - loss: 0.3486 - val_accuracy: 0.8774 - val_loss: 0.3427
Epoch 6/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8811 - loss: 0.3333 - val_accuracy: 0.8800 - val_loss: 0.3449
Epoch 7/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8848 - loss: 0.3233 - val_accuracy: 0.8806 - val_loss: 0.3371
Epoch 8/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8891 - loss: 0.3118 -

#### Application to CIFAR10

In [16]:
# Import CIFAR10 dataset
CIFAR10 = tf.keras.datasets.cifar10


# Preprocess data
(X_train_full, y_train_full), (X_test, y_test) = CIFAR10.load_data()
print(X_train_full.shape, y_train_full.shape)

X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_test = X_test / 255.0
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)


# Choose hyperparameters
seed = 900
activation_function = 'leaky_relu'

kernel_ini = "glorot_uniform"
bias_ini = keras.initializers.Zeros()
shape = shape = [32,32,3]
learning_rate = 0.001


# Build model
model1 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=shape),
    keras.layers.Dense(400, kernel_initializer = kernel_ini,
    activation=activation_function),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(200, kernel_initializer = kernel_ini,
    activation=activation_function),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(100, kernel_initializer = kernel_ini,
    activation=activation_function, kernel_regularizer=kernel_reg),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(50, kernel_initializer = kernel_ini,
    activation=activation_function),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(10, kernel_initializer = kernel_ini,
    activation="softmax")
])
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True
)

model1.summary()


model1.compile(loss="sparse_categorical_crossentropy",
optimizer=keras.optimizers.AdamW(learning_rate=learning_rate),
metrics=["accuracy",
        #   tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
          ])
model1.fit(
    X_train, y_train,
    epochs=50,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping]
)

# Evaluate the model on the test set
test_loss, test_acc = model1.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

(50000, 32, 32, 3) (50000, 1)
(45000, 32, 32, 3) (45000, 1)
(5000, 32, 32, 3) (5000, 1)


/vol/home/s2358441/.conda/envs/IDL/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_12 (Flatten)            │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_60 (Dense)                │ (None, 400)            │     1,229,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_48 (Dropout)            │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_61 (Dense)                │ (None, 200)            │        80,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_49 (Dropout)            │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_62 (Dense)                │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_50 (Dropout)            │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_63 (Dense)                │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_51 (Dropout)            │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ (None, 10)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,335,060 (5.09 MB)

 Trainable params: 1,335,060 (5.09 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.2809 - loss: 1.9849 - val_accuracy: 0.3258 - val_loss: 1.8354
Epoch 2/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.3456 - loss: 1.8256 - val_accuracy: 0.3820 - val_loss: 1.7249
Epoch 3/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.3766 - loss: 1.7497 - val_accuracy: 0.3930 - val_loss: 1.6801
Epoch 4/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4007 - loss: 1.6823 - val_accuracy: 0.4160 - val_loss: 1.6257
Epoch 5/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4142 - loss: 1.6360 - val_accuracy: 0.4292 - val_loss: 1.5956
Epoch 6/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4305 - loss: 1.5990 - val_accuracy: 0.4280 - val_loss: 1.5926
Epoch 7/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4408 - loss: 1.5729 - val_accuracy: 0.4350 - val_loss: 1.5696
Epoch 8/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4507 - loss: 1.5459 -


### Model 3

In [ ]:
# Import FashionMNIST data
fashion_mnist= keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
print(X_train_full.shape, y_train_full.shape)


# Preprocess data
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_test = X_test / 255.0
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)


# Choose hyperparameters
seed = 900
activation_function = 'leaky_relu'

kernel_ini = keras.initializers.Orthogonal(gain = 1.0, seed = seed)
bias_ini = keras.initializers.Zeros()
shape = [28, 28]
learning_rate = 0.001


# Build model
model1 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=shape),
    keras.layers.Dense(400, kernel_initializer = kernel_ini,
    activation=activation_function),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(200, kernel_initializer = kernel_ini,
    activation=activation_function),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(100, kernel_initializer = kernel_ini,
    activation=activation_function, kernel_regularizer=kernel_reg),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(50, kernel_initializer = kernel_ini,
    activation=activation_function),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(10, kernel_initializer = kernel_ini,
    activation="softmax")
])
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True
)

model1.summary()


model1.compile(loss="sparse_categorical_crossentropy",
optimizer=keras.optimizers.Nadam(learning_rate=learning_rate),
metrics=["accuracy",
        #   tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
          ])
model1.fit(
    X_train, y_train,
    epochs=50,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping]
)

# Evaluate the model on the test set
test_loss, test_acc = model1.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)